In [13]:
import numpy as np
import pandas as pd

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import LSTM



# Packages for data preparation
# from sklearn.model_selection import train_test_split # Dont need because data folder has already spilt
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder


#For Training Multinomial Naive Bayess
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV


In [7]:
data = pd.read_csv("sa-emotions/train_data.csv")
data.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


## Text Preprocessing

In [12]:
import re

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words


'''
By default, all punctuation is removed, turning the texts into space-separated sequences of words
'''
# def remove_punctuation(words):
#     """Remove punctuation from list of tokenized words"""
#     new_words = []
#     for word in words:
#         new_word = re.sub(r'[^\w\s]', '', word)
#         if new_word != '':
#             new_words.append(new_word)
#     return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems


def remove_mentions(words):
    return re.sub(r'@[A-Za-z0-9]+','', words)


def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = replace_numbers(words)
    words = remove_stopwords(words)
    return words



# tweets = [normalize(tweet) for tweet in tweets]
    
    

'neutral'

### Keras Tokenizer will convert the tweets to sequences so that they can be passed through embedding matrices.

In [ ]:
tk = Tokenizer(lower = True, filters='')
tk.fit_on_texts(full_text)

train_tokenized = tk.texts_to_sequences(train['tweet'])
test_tokenized = tk.texts_to_sequences(test['tweet'])

max_len = 50
X_train = pad_sequences(train_tokenized, maxlen = max_len)
X_test = pad_sequences(test_tokenized, maxlen = max_len)

**fit_on_texts** Updates internal vocabulary based on a list of texts. This method creates the vocabulary index based on word frequency. So if you give it something like, "The cat sat on the mat." It will create a dictionary s.t. word_index["the"] = 1; word_index["cat"] = 2 it is word -> index dictionary so every word gets a unique integer value. 0 is reserved for padding. So lower integer means more frequent word (often the first few are stop words because they appear a lot).

**texts_to_sequences** Transforms each text in texts to a sequence of integers. So it basically takes each word in the text and replaces it with its corresponding integer value from the word_index dictionary. Nothing more, nothing less, certainly no magic involved.

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texs(all_tweets)
train_tokenized = tk.texts_to_sequences(X_train)

max_length = 70

X_train = pad_sequences(train_tokenized, maxlen = max_len)

## Data Preprocessing Notes
* Need to remove mentions


 ### Medium Research Notes:

If we need to create a embedding mapping for words with training, how does this affect how we embed the test data (streamed tweets)? Maybe just use Glove.

Shall we use Embedding layer as our input layer?

Preprocessing
KD article
Padding
Remove stop words


CNN training is 1.5 times faster than training LSTMs

Need to determine the unique vocabulary of training in order to determine embedding vector length, then I think you pad off of that? Not sure about the padding preprocessing step.

Pathtoword embeddings is the path where we’ve  downloaded word embeddings via glove or number batch

Is it cheating if I understand all the processing of what this article is doing, and add my own preprocessing, and maybe additional steps? 
https://medium.com/@panghalarsh/sentiment-analysis-in-python-using-keras-glove-twitter-word-embeddings-and-deep-rnn-on-a-combined-580646cb900a

Word Embeddings is better than count vectorizers that one hot encodes sentences because the bagofwords model will not scale well to large datasets due to create sparse vectors in high dimensionality. Also we lose semantics through BagofWords approach. Word embeddings are dense vectors with much lower dimensionality. Secondly, the semantic relationships between words are reflected in the distance and direction of the vectors.

Should read into each Keras library being utilized and understanding 


Need to encode output classes (emotions) w/ LabelEncoders

Why do we need to Flatten after we do embedding? Like what does that do?

In the Embedding layer (which is layer 0 here) we set the weights for the words to those found in the GloVe word embeddings. By setting trainable to False we make sure that the GloVe word embeddings cannot be changed.

The best result is achieved with 100-dimensional word embeddings that are trained on the available data. By doing this, we do not take into account the relationships between the words in the tweet. This can be achieved with a recurrent neural network or a 1D convolutional network. But that’s something for a future post.
— Bert Carremans 


Do we need padding when utilizing word embeddings? I saw some Word2Vec tutorials using padding one hot encoded sequences, but my understanding was that we don’t need it for embeddings.
https://medium.com/datadriveninvestor/sentiment-analysis-using-embeddings-f3dd99aeaade


Why train my own word embeddings at all? Why not always use Glove? It seems to be a better trained for my model’s embedding layer anyways.


There are so many state of the art embedding techniques like SkipGrams, FastText, and ELMo. I should investigate these for their result to text summarizations.
